In [ ]:
import os
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from tensorflow.keras import layers, models
from tensorflow.keras.optimizers import Adam

# Path to the folder containing images
good_tyre_path = '/kaggle/input/tyre-quality-classification/Digital images of defective and good condition tyres/good'
bad_tyre_path = '/kaggle/input/tyre-quality-classification/Digital images of defective and good condition tyres/defective'

# Function to load and preprocess images
def load_and_preprocess_images(folder_path, label):
    images = []
    labels = []
    for filename in os.listdir(folder_path):
        img_path = os.path.join(folder_path, filename)
        img = load_img(img_path, target_size=(img_height, img_width))
        img_array = img_to_array(img)
        images.append(img_array)
        labels.append(label)
    return np.array(images), np.array(labels)

# Load and preprocess images
img_height, img_width = 128, 128
good_tyre_images, good_tyre_labels = load_and_preprocess_images(good_tyre_path, label=0)
bad_tyre_images, bad_tyre_labels = load_and_preprocess_images(bad_tyre_path, label=1)

# Combine and shuffle the data
all_images = np.concatenate((good_tyre_images, bad_tyre_images), axis=0)
all_labels = np.concatenate((good_tyre_labels, bad_tyre_labels), axis=0)
shuffle_indices = np.random.permutation(len(all_images))
all_images = all_images[shuffle_indices]
all_labels = all_labels[shuffle_indices]

# Split the data into training and validation sets
split_ratio = 0.8
split_index = int(len(all_images) * split_ratio)

train_data, train_labels = all_images[:split_index], all_labels[:split_index]
val_data, val_labels = all_images[split_index:], all_labels[split_index:]

# Data Augmentation
datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Build the CNN model
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(img_height, img_width, 3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Flatten())
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(1, activation='sigmoid'))

# Compile the model with a lower learning rate
model.compile(optimizer=Adam(learning_rate=0.0001), loss='binary_crossentropy', metrics=['accuracy'])

# Train the model for more epochs
model.fit(datagen.flow(train_data, train_labels, batch_size=32), epochs=50, validation_data=(val_data, val_labels))

# Save the model
model.save('improved_cnn_model.h5')




In [ ]:
import os
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models
from tensorflow.keras.applications import VGG16
from tensorflow.keras.applications.vgg16 import preprocess_input

# Path to the folder containing images
good_tyre_path = '/kaggle/input/tyre-quality-classification/Digital images of defective and good condition tyres/good'
bad_tyre_path = '/kaggle/input/tyre-quality-classification/Digital images of defective and good condition tyres/defective'

# Function to load and preprocess images
def load_and_preprocess_images(folder_path, label):
    images = []
    labels = []
    for filename in os.listdir(folder_path):
        img_path = os.path.join(folder_path, filename)
        img = load_img(img_path, target_size=(img_height, img_width))
        img_array = img_to_array(img)
        images.append(img_array)
        labels.append(label)
    return np.array(images), np.array(labels)

# Load and preprocess images
img_height, img_width = 128, 128
good_tyre_images, good_tyre_labels = load_and_preprocess_images(good_tyre_path, label=0)
bad_tyre_images, bad_tyre_labels = load_and_preprocess_images(bad_tyre_path, label=1)

# Combine and shuffle the data
all_images = np.concatenate((good_tyre_images, bad_tyre_images), axis=0)
all_labels = np.concatenate((good_tyre_labels, bad_tyre_labels), axis=0)
shuffle_indices = np.random.permutation(len(all_images))
all_images = all_images[shuffle_indices]
all_labels = all_labels[shuffle_indices]

# Split the data into training and validation sets
split_ratio = 0.8
split_index = int(len(all_images) * split_ratio)

train_data, train_labels = all_images[:split_index], all_labels[:split_index]
val_data, val_labels = all_images[split_index:], all_labels[split_index:]

# Data Augmentation
datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Load the VGG16 model with pre-trained weights
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(img_height, img_width, 3))

# Freeze the pre-trained layers
for layer in base_model.layers:
    layer.trainable = False

# Create a new model on top
model = models.Sequential()
model.add(base_model)
model.add(layers.Flatten())
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(1, activation='sigmoid'))

# Compile the model with binary crossentropy
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model with data augmentation
model.fit(datagen.flow(train_data, train_labels, batch_size=32), epochs=20, validation_data=(val_data, val_labels))

# Save the model in .h5 format
model.save('transfer_learning_model.h5')

# Evaluation on folders
def evaluate_folder(folder_path, model):
    normal_count = 0
    cracked_count = 0

    for filename in os.listdir(folder_path):
        img_path = os.path.join(folder_path, filename)

        if os.path.isfile(img_path) and img_path.lower().endswith(('.png', '.jpg', '.jpeg')):
            img_array = load_and_preprocess_image(img_path)
            img_array = preprocess_input(img_array)
            img_array = np.expand_dims(img_array, axis=0)

            prediction = model.predict(img_array)[0, 0]
            predicted_class = 1 if prediction > 0.5 else 0

            if predicted_class == 1:
                cracked_count += 1
            else:
                normal_count += 1

    print(f"Total Cracked Tires: {cracked_count}")
    print(f"Total Normal Tires: {normal_count}")




    

In [9]:
import os
import numpy as np
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import load_img, img_to_array

# Path to the folder containing images to be predicted
prediction_folder = '/kaggle/input/tire-texture-image-recognition/Tire Textures/training_data/normal'

# Path to the saved model
model_path = '/kaggle/working/improved_cnn_model_v4.h5'  # Change this to the path where your saved model is located

# Function to load and preprocess a single image
def load_and_preprocess_image(img_path, target_size=(128, 128)):
    img = load_img(img_path, target_size=target_size)
    img_array = img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    return img_array

# Load the saved model
model = load_model(model_path)

# Counters for normal and cracked predictions
normal_count = 0
cracked_count = 0

error = []
# Iterate through images in the prediction folder
for filename in os.listdir(prediction_folder):
    img_path = os.path.join(prediction_folder, filename)

    # Check if the file is a valid image file
    if os.path.isfile(img_path) and img_path.lower().endswith(('.png', '.jpg', '.jpeg')):
        # Load and preprocess the image
        img_array = load_and_preprocess_image(img_path)

        # Make predictions
        prediction = model.predict(img_array)[0, 0]  # Extracting the scalar value from the prediction tensor

        # Determine predicted class
        predicted_class = 1 if prediction > 0.5 else 0

        # Update counters
        if predicted_class == 1:
            cracked_count += 1
            error.append(prediction)
            
        else:
            normal_count += 1
            

# Display counts
print(f"Total Cracked Tires: {cracked_count}")
print(f"Total Normal Tires: {normal_count}")

print(error)




1/1 [==============================] - 0s 20ms/step
Total Cracked Tires: 27
Total Normal Tires: 349
[0.82300955, 0.9428467, 0.5190063, 0.60807484, 0.99555475, 0.8346801, 0.99044436, 0.8144256, 0.7628027, 0.68929857, 0.8948047, 0.9924213, 0.99902093, 0.9894162, 0.94532424, 0.58423644, 0.57446885, 0.7167496, 0.88265693, 0.98301053, 0.99466383, 0.91518044, 0.9106118, 0.5904066, 0.69559973, 0.6350307, 0.6202444]


In [5]:
sumn = 0
for i in range(len(error)):
    sumn += error[i]
    
    
sumn / len(error)


0.9064351875028569

In [1]:
import os
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from tensorflow.keras import layers, models
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import LearningRateScheduler

# Path to the folder containing images
good_tyre_path = '/kaggle/input/tyre-quality-classification/Digital images of defective and good condition tyres/good'
bad_tyre_path = '/kaggle/input/tyre-quality-classification/Digital images of defective and good condition tyres/defective'

# Function to load and preprocess images
def load_and_preprocess_images(folder_path, label):
    images = []
    labels = []
    for filename in os.listdir(folder_path):
        img_path = os.path.join(folder_path, filename)
        img = load_img(img_path, target_size=(img_height, img_width))
        img_array = img_to_array(img)
        images.append(img_array)
        labels.append(label)
    return np.array(images), np.array(labels)

# Load and preprocess images
img_height, img_width = 128, 128
good_tyre_images, good_tyre_labels = load_and_preprocess_images(good_tyre_path, label=0)
bad_tyre_images, bad_tyre_labels = load_and_preprocess_images(bad_tyre_path, label=1)

# Combine and shuffle the data
all_images = np.concatenate((good_tyre_images, bad_tyre_images), axis=0)
all_labels = np.concatenate((good_tyre_labels, bad_tyre_labels), axis=0)
shuffle_indices = np.random.permutation(len(all_images))
all_images = all_images[shuffle_indices]
all_labels = all_labels[shuffle_indices]

# Split the data into training and validation sets
split_ratio = 0.8
split_index = int(len(all_images) * split_ratio)

train_data, train_labels = all_images[:split_index], all_labels[:split_index]
val_data, val_labels = all_images[split_index:], all_labels[split_index:]

# Data Augmentation
datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Use a learning rate scheduler
def lr_schedule(epoch):
    lr = 0.00001
    if epoch > 60:
        lr *= 0.1
    return lr

lr_scheduler = LearningRateScheduler(lr_schedule)

# Deeper Model with Batch Normalization and Increased Dropout
model = models.Sequential()
model.add(layers.Conv2D(64, (3, 3), activation='relu', input_shape=(img_height, img_width, 3)))
model.add(layers.BatchNormalization())
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.BatchNormalization())
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(256, (3, 3), activation='relu'))
model.add(layers.BatchNormalization())
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Flatten())
model.add(layers.Dense(512, activation='relu'))
model.add(layers.Dropout(0.7))
model.add(layers.Dense(1, activation='sigmoid'))

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.00001), loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(datagen.flow(train_data, train_labels, batch_size=32), epochs=120,  
          validation_data=(val_data, val_labels), callbacks=[lr_scheduler])

# Save the model
model.save('improved_cnn_model_v3.h5')


/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


Epoch 1/120
47/47 [==============================] - 13s 153ms/step - loss: 1.3249 - accuracy: 0.5472 - val_loss: 1.0299 - val_accuracy: 0.6102 - lr: 1.0000e-05
Epoch 2/120
47/47 [==============================] - 7s 145ms/step - loss: 1.1071 - accuracy: 0.6119 - val_loss: 0.7151 - val_accuracy: 0.6371 - lr: 1.0000e-05
Epoch 3/120
47/47 [==============================] - 7s 144ms/step - loss: 0.9771 - accuracy: 0.6233 - val_loss: 0.6433 - val_accuracy: 0.7070 - lr: 1.0000e-05
Epoch 4/120
47/47 [==============================] - 7s 142ms/step - loss: 0.9183 - accuracy: 0.6368 - val_loss: 0.6393 - val_accuracy: 0.7151 - lr: 1.0000e-05
Epoch 5/120
47/47 [==============================] - 7s 143ms/step - loss: 0.8552 - accuracy: 0.6617 - val_loss: 0.6625 - val_accuracy: 0.6962 - lr: 1.0000e-05
Epoch 6/120
47/47 [==============================] - 7s 142ms/step - loss: 0.7706 - accuracy: 0.6772 - val_loss: 0.6116 - val_accuracy: 0.7446 - lr: 1.0000e-05
Epoch 7/120
47/47 [====================

KeyboardInterrupt: 

In [ ]:
import os

os.remove('/kaggle/working/improved_cnn_model.h5')

In [6]:
import os
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from tensorflow.keras import layers, models
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import LearningRateScheduler

# Path to the folder containing images
good_tyre_path = '/kaggle/input/tyre-quality-classification/Digital images of defective and good condition tyres/good'
bad_tyre_path = '/kaggle/input/tyre-quality-classification/Digital images of defective and good condition tyres/defective'

# Function to load and preprocess images
def load_and_preprocess_images(folder_path, label, target_size=(128, 128)): 
    images = []
    labels = []
    for filename in os.listdir(folder_path):
        img_path = os.path.join(folder_path, filename)
        img = load_img(img_path, target_size=target_size)
        img_array = img_to_array(img)
        images.append(img_array)
        labels.append(label)
    return np.array(images), np.array(labels)

# Load and preprocess images
good_tyre_images, good_tyre_labels = load_and_preprocess_images(good_tyre_path, label=0)
bad_tyre_images, bad_tyre_labels = load_and_preprocess_images(bad_tyre_path, label=1)

# Combine and shuffle the data
all_images = np.concatenate((good_tyre_images, bad_tyre_images), axis=0)
all_labels = np.concatenate((good_tyre_labels, bad_tyre_labels), axis=0)
shuffle_indices = np.random.permutation(len(all_images))
all_images = all_images[shuffle_indices]
all_labels = all_labels[shuffle_indices]

# Split the data into training and validation sets
split_ratio = 0.8
split_index = int(len(all_images) * split_ratio)

train_data, train_labels = all_images[:split_index], all_labels[:split_index]
val_data, val_labels = all_images[split_index:], all_labels[split_index:]

# Data Augmentation
datagen = ImageDataGenerator(
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Use a learning rate scheduler
def lr_schedule(epoch):
    lr = 0.0001
    if epoch > 60:
        lr *= 0.1
    return lr

lr_scheduler = LearningRateScheduler(lr_schedule)

# Deeper Model with Batch Normalization and Increased Dropout
model = models.Sequential()
model.add(layers.Conv2D(64, (3, 3), activation='relu', input_shape=(128, 128, 3)))
model.add(layers.BatchNormalization())

model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.BatchNormalization())

model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(256, (3, 3), activation='relu'))
model.add(layers.BatchNormalization())

model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Flatten())

model.add(layers.Dense(512, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(1, activation='sigmoid'))

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.0001), loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(datagen.flow(train_data, train_labels, batch_size=32), epochs=120,  
          validation_data=(val_data, val_labels), callbacks=[lr_scheduler])

# Save the model
model.save('dude.h5')


Epoch 1/120
47/47 [==============================] - 10s 149ms/step - loss: 1.7143 - accuracy: 0.6220 - val_loss: 0.6594 - val_accuracy: 0.7097 - lr: 1.0000e-04
Epoch 2/120
47/47 [==============================] - 7s 143ms/step - loss: 1.0904 - accuracy: 0.6449 - val_loss: 0.5987 - val_accuracy: 0.7312 - lr: 1.0000e-04
Epoch 3/120
47/47 [==============================] - 7s 141ms/step - loss: 0.6887 - accuracy: 0.6853 - val_loss: 0.5318 - val_accuracy: 0.7419 - lr: 1.0000e-04
Epoch 4/120
47/47 [==============================] - 7s 140ms/step - loss: 0.6107 - accuracy: 0.7136 - val_loss: 0.4995 - val_accuracy: 0.7957 - lr: 1.0000e-04
Epoch 5/120
47/47 [==============================] - 7s 142ms/step - loss: 0.5920 - accuracy: 0.7156 - val_loss: 0.4693 - val_accuracy: 0.7823 - lr: 1.0000e-04
Epoch 6/120
47/47 [==============================] - 7s 143ms/step - loss: 0.5520 - accuracy: 0.7358 - val_loss: 0.5623 - val_accuracy: 0.7446 - lr: 1.0000e-04
Epoch 7/120
47/47 [====================

In [1]:
import numpy as np
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import load_img, img_to_array

# Path to the saved model
model_path = 'dude.h5'  # Change this to the path where your saved model is located

# Function to load and preprocess a single image
def load_and_preprocess_image(img_path, target_size=(128, 128)):
    img = load_img(img_path, target_size=target_size)
    img_array = img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    return img_array

# Load the saved model
model = load_model(model_path)

# Path to the image you want to predict
image_path = '/home/karthik/Desktop/CNN/catdog/dataset/training_set/normal/NewNormal62.jpg'

# Load and preprocess the image
img_array = load_and_preprocess_image(image_path)

# Make prediction
prediction = model.predict(img_array)[0, 0]  # Extracting the scalar value from the prediction tensor

# Determine predicted class
predicted_class = 1 if prediction > 0.5 else 0

# Display the prediction result
if predicted_class == 1:
    print("Predicted: Cracked Tire")
else:
    print("Predicted: Normal Tire")

print(prediction)

2024-03-05 03:20:44.598693: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-03-05 03:20:44.776815: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-05 03:20:44.776942: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-05 03:20:44.803356: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-03-05 03:20:44.867914: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-03-05 03:20:44.869247: I tensorflow/core/platform/cpu_feature_guard.cc:1

1/1 [==============================] - 0s 168ms/step
Predicted: Normal Tire
3.0470113e-10
